## Naive Bayes

### data of solo 

In [1]:
from __future__ import print_function
%matplotlib inline
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy as sp
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.cross_validation import KFold, cross_val_score, train_test_split
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

/Users/minhu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data=pd.read_csv("PUBG_player.csv",encoding='ISO-8859-1')
data.head()

,player_name,tracker_id,solo_KillDeathRatio,solo_WinRatio,solo_RoundsPlayed,solo_Wins,solo_Rating,solo_DamagePg,solo_HealsPg,solo_KillsPg,...,squad_HealsPg,squad_KillsPg,squad_MoveDistancePg,squad_RevivesPg,squad_RoundMostKills,squad_AvgSurvivalTime,squad_WinPoints,squad_AvgWalkDistance,squad_AvgRideDistance,squad_TimeSurvivedPg
0,BreakNeck,4405,3.14,17.65,17,3,1559.78,255.36,1.94,2.59,...,3.40,2.86,8091.57,0.36,11,1449.03,4868,2626.97,4372.64,1657.95
1,Blackwalk,8199,4.41,18.18,33,6,1884.53,393.04,1.82,3.61,...,2.83,3.05,8381.97,0.38,14,1570.51,4058,2422.48,6009.73,1630.18
2,mercedes_benz,4454,3.60,0.00,5,0,1255.92,329.76,1.00,3.60,...,2.41,3.20,7625.66,0.29,17,1223.28,5929,1871.89,3011.87,1591.65
3,DORA,7729,14.00,50.00,8,4,1798.70,751.95,1.00,7.00,...,2.47,2.76,7733.03,0.29,12,1583.73,2576,2154.62,5578.41,1583.73
4,n2tstar,0,10.50,33.33,6,2,1668.67,637.48,2.00,7.00,...,2.99,3.05,7741.01,0.29,15,1608.96,2579,2098.47,5642.54,1608.96


In [3]:
data_solo=data[['solo_Wins','solo_DamagePg','solo_HealsPg','solo_KillsPg','solo_WinRatio']]
data_solo.head()

,solo_Wins,solo_DamagePg,solo_HealsPg,solo_KillsPg,solo_WinRatio
0,3,255.36,1.94,2.59,17.65
1,6,393.04,1.82,3.61,18.18
2,0,329.76,1.00,3.60,0.00
3,4,751.95,1.00,7.00,50.00
4,2,637.48,2.00,7.00,33.33


In [4]:
data_solo.head(n=3000).to_csv('solo_bayes.csv',index=False)
data_solo = data_solo.head(n=3000)

In [5]:
# for i in range(len(pre_dataY)):
#     if pre_dataY[i] >= 15:
#         dataY[i][0] = 1
#     else:
#         dataY[i][1] = 1
length=len(data_solo['solo_Wins'])
length
for i in range(length):
    if data_solo['solo_WinRatio'][i]>=25:
        data_solo['solo_WinRatio'][i]=1
    else:
        data_solo['solo_WinRatio'][i]=0

In [6]:
data_solo.head(n=10)

,solo_Wins,solo_DamagePg,solo_HealsPg,solo_KillsPg,solo_WinRatio
0,3,255.36,1.94,2.59,1.0
1,6,393.04,1.82,3.61,1.0
2,0,329.76,1.00,3.60,0.0
3,4,751.95,1.00,7.00,1.0
4,2,637.48,2.00,7.00,1.0
5,5,411.95,1.38,3.94,1.0
6,12,284.57,1.94,2.24,0.0
7,5,238.40,1.53,2.00,1.0
8,6,339.44,1.52,3.18,0.0
9,2,372.38,1.12,3.12,1.0


In [7]:

# apply the ceiling function to the numeric columns
# iris.loc[:, 'sepal_length':'petal_width'] = iris.loc[:, 'sepal_length':'petal_width'].apply(np.ceil)
# iris.head()

# apply the ceiling function to the numeric columns
data_solo.loc[:, 'solo_DamagePg':'solo_KillsPg'] = data_solo.loc[:,'solo_DamagePg':'solo_KillsPg'].apply(np.ceil)
data_solo.head()

,solo_Wins,solo_DamagePg,solo_HealsPg,solo_KillsPg,solo_WinRatio
0,3,256.0,2.0,3.0,1.0
1,6,394.0,2.0,4.0,1.0
2,0,330.0,1.0,4.0,0.0
3,4,752.0,1.0,7.0,1.0
4,2,638.0,2.0,7.0,1.0


In [8]:
data_solo.describe()

,solo_Wins,solo_DamagePg,solo_HealsPg,solo_KillsPg,solo_WinRatio
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,6.443000,257.427333,2.134333,2.735333,0.171667
std,9.735965,131.555642,0.991606,1.315242,0.377153
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,171.000000,2.000000,2.000000,0.000000
50%,3.000000,232.000000,2.000000,2.000000,0.000000
75%,7.000000,313.000000,2.000000,3.000000,0.000000
max,106.000000,1569.000000,13.000000,16.000000,1.000000


In [9]:
# create X and y
X = data_solo.drop('solo_WinRatio', axis=1)
y = data_solo.solo_WinRatio

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [11]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred_class))

0.814666666667


In [12]:
# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred_class))

0.845333333333


### data of double players

In [13]:
data_double = data[['duo_Wins','duo_KillDeathRatio','duo_DamagePg','duo_HealsPg','duo_KillsPg','duo_WinRatio']]
data_double.head()

,duo_Wins,duo_KillDeathRatio,duo_DamagePg,duo_HealsPg,duo_KillsPg,duo_WinRatio
0,5,6.50,569.60,2.33,4.33,33.33
1,5,4.56,338.88,1.93,2.93,35.71
2,6,6.55,527.80,1.65,4.24,35.29
3,2,8.00,434.13,0.33,2.67,66.67
4,27,5.03,428.62,1.90,3.73,25.71


In [14]:
data_double = data_double.head(n=3000)
length

3000

In [15]:
for i in range(length):
    if data_double['duo_WinRatio'][i]>=15:
        data_double['duo_WinRatio'][i]=1
    else:
        data_double['duo_WinRatio'][i]=0

In [16]:
data_double.head(n=10)

,duo_Wins,duo_KillDeathRatio,duo_DamagePg,duo_HealsPg,duo_KillsPg,duo_WinRatio
0,5,6.50,569.60,2.33,4.33,1.0
1,5,4.56,338.88,1.93,2.93,1.0
2,6,6.55,527.80,1.65,4.24,1.0
3,2,8.00,434.13,0.33,2.67,1.0
4,27,5.03,428.62,1.90,3.73,1.0
5,11,2.94,305.34,1.87,2.51,0.0
6,38,3.46,355.47,3.13,2.66,1.0
7,5,2.82,288.64,2.26,2.30,1.0
8,3,3.16,339.02,2.95,2.73,0.0
9,3,4.00,370.03,1.47,3.29,1.0


In [18]:
X=data_double.drop('duo_WinRatio', axis=1)
y=data_double.duo_WinRatio

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [20]:

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred_class))

0.744


In [21]:

# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred_class))

0.873333333333


### data of squad players

In [23]:

data_squad=data[['squad_Wins','squad_KillDeathRatio','squad_DamagePg','squad_HealsPg','squad_KillsPg','squad_Rating','squad_WinRatio']]
data_squad.head()

,squad_Wins,squad_KillDeathRatio,squad_DamagePg,squad_HealsPg,squad_KillsPg,squad_Rating,squad_WinRatio
0,305,5.45,377.15,3.40,2.86,3212.25,47.51
1,338,5.73,373.68,2.83,3.05,3156.94,46.81
2,347,6.08,399.70,2.41,3.20,3170.41,47.34
3,207,4.77,368.85,2.47,2.76,3078.62,42.16
4,193,5.70,384.69,2.99,3.05,3085.86,46.39


In [24]:
data_squad = data_squad.head(n=3000)
length = len(data_squad['squad_WinRatio'])

In [25]:
for i in range(length):
    if data_squad['squad_WinRatio'][i]>=35:
        data_squad['squad_WinRatio'][i]=1
    else:
        data_squad['squad_WinRatio'][i]=0

In [26]:
data_squad.head()

,squad_Wins,squad_KillDeathRatio,squad_DamagePg,squad_HealsPg,squad_KillsPg,squad_Rating,squad_WinRatio
0,305,5.45,377.15,3.40,2.86,3212.25,1.0
1,338,5.73,373.68,2.83,3.05,3156.94,1.0
2,347,6.08,399.70,2.41,3.20,3170.41,1.0
3,207,4.77,368.85,2.47,2.76,3078.62,1.0
4,193,5.70,384.69,2.99,3.05,3085.86,1.0


In [27]:
X=data_squad.drop('squad_WinRatio',axis=1)
y= data_squad.squad_WinRatio

In [28]:
X_train,X_test,y_train, y_test = train_test_split(X,y,random_state=1)

In [29]:

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred_class = mnb.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred_class))

0.882666666667


In [30]:
# testing accuracy of Gaussian Naive Bayes
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred_class = gnb.predict(X_test)
print (metrics.accuracy_score(y_test, y_pred_class))

0.929333333333
